In [34]:
# Importing dependencies
import pandas as pd
import numpy as np
from collections import defaultdict

# for visualizations
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

# For Hypothesis testing
import statsmodels.formula.api as smf

# For principal components analysis
from sklearn.decomposition import PCA

### Loading the Dataset

In [3]:
# Loading the dataset into pandas dataframe
path = "../data/census-income.data.gz"
# set the column names
censusColnames = ['Age', 'ClassOfWorker', 'Industry', 'Occupation', 'Education',
                  'WagePerHr', 'EducationalInst', 'MaritalStatus', 'IndustryCode', 
                  'OccupationCode', 'Race', 'HispanicOrigin', 'Sex', 'MemLabourUnion',
                  'UnemploymentReason', 'EmploymentStatus', 'CapitalGain', 'CapitalLoss',
                  'Dividends', 'FEDERALTAX', 'TaxFilerStat', 'PrevState', 
                  'HouseholdStatus', 'HouseholdSummary', 'INSTANCEWEIGHT', 
                  'MigrationCode_MSA', 'MigrationCode_REG', 
                  'MigrationCode_WITHIN_REG', 'HouseOneYearAgo', 
                  'MigrationPrevResInSunbelt', 'NumOfPersonForEmployer', 'Parent', 
                  'BirthCountryFather', 'BirthCountryMother', 'BirthCountrySelf', 
                  'Citizenship', 'OwnBusiness', 'VeteranQA', 'VeteranBenefits', 
                  'WeeksWorked', 'Year', 'targetIncome']
censusDf = pd.read_csv(path, sep=r',', skipinitialspace=True, 
                       names = censusColnames, header='infer')

# Printing the dimensions of the dataset
print(censusDf.shape[0],"rows,", censusDf.shape[1],"columns")

# Displaying first five elements of all columns
with pd.option_context('display.max_columns', None):
    display(censusDf.head())

199523 rows, 42 columns


,Age,ClassOfWorker,Industry,Occupation,Education,WagePerHr,EducationalInst,MaritalStatus,IndustryCode,OccupationCode,Race,HispanicOrigin,Sex,MemLabourUnion,UnemploymentReason,EmploymentStatus,CapitalGain,CapitalLoss,Dividends,FEDERALTAX,TaxFilerStat,PrevState,HouseholdStatus,HouseholdSummary,INSTANCEWEIGHT,MigrationCode_MSA,MigrationCode_REG,MigrationCode_WITHIN_REG,HouseOneYearAgo,MigrationPrevResInSunbelt,NumOfPersonForEmployer,Parent,BirthCountryFather,BirthCountryMother,BirthCountrySelf,Citizenship,OwnBusiness,VeteranQA,VeteranBenefits,WeeksWorked,Year,targetIncome
0,73,Not in universe,0,0,High school graduate,0,Not in universe,Widowed,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Other Rel 18+ ever marr not in subfamily,Other relative of householder,1700.09,?,?,?,Not in universe under 1 year old,?,0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,- 50000.
1,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Head of household,South,Arkansas,Householder,Householder,1053.55,MSA to MSA,Same county,Same county,No,Yes,1,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,- 50000.
2,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,Asian or Pacific Islander,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Child 18+ never marr Not in a subfamily,Child 18 or older,991.95,?,?,?,Not in universe under 1 year old,?,0,Not in universe,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,- 50000.
3,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1758.14,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.
4,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1069.16,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.


---

## Problem Statement

>From the various features in the census data set our aim is to build a predictive model to determine whether the income level for the people in United States exceeds the bracket of $50,000.

## Hypothesis Generation

From our problem statement is clear that it is a binary classification problem.

Let us generate some hypotheses which will help us in building the models more efficiently. We need to figure out some hypotheses which might influence our final outcome, hence we need to answer a simple question.

**Is There a Relationship Between the Response and Predictors?**

To test this we use the test between the Null Hypothesis $H_0$ versus the Alternate Hypothesis $H_a$.
* $H_0$ : There is no relationship between the response Income and the predictors.
    * To test the Null Hypothesis we test whether all the regression coefficients are zero.
* $H_a$ : There is some realtionship between the response and the predictors.
    * To test the Alternate Hypothesis we find  at least one coefficient that is non-zero.
    
*To perform the Hypothesis tests we will be performing multivariate linear regression on ordinal values of the dataset using **statsmodels** library.*


In [4]:
# create a dataframe for hypothesis testing
censusDf_htest = censusDf
censusDf_htest['targetIncome'] = pd.get_dummies(censusDf_htest.targetIncome).iloc[:,1:]

# Constructing a linearmodel using the ordinal values for our initial hypothesis test
hypothesis_test_model = smf.ols(formula=("targetIncome ~ Age + Industry + Occupation + "
             "WagePerHr + CapitalGain + CapitalLoss + Dividends + "
             "INSTANCEWEIGHT + NumOfPersonForEmployer + OwnBusiness +"
             "VeteranBenefits + WeeksWorked + Year"), data=censusDf_htest).fit()

# Printing the summary of the model
hypothesis_test_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           targetIncome   R-squared:                       0.195
Model:                            OLS   Adj. R-squared:                  0.195
Method:                 Least Squares   F-statistic:                     3710.
Date:                Sat, 18 Nov 2017   Prob (F-statistic):               0.00
Time:                        13:42:47   Log-Likelihood:                 22185.
No. Observations:              199523   AIC:                        -4.434e+04
Df Residuals:                  199509   BIC:                        -4.420e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.4296      0.092     -4.688      0.000      -0.609      -0.250
Age                        0.0010   3.15e-05     32.642      0.000       0.001       0.001
Industry                   0.0004   4.23e-05      9.753      0.000       0.000       0.000
Occupation                -0.0040   4.61e-05    -86.916      0.000      -0.004      -0.004
WagePerHr              -8.225e-06   1.81e-06     -4.545      0.000   -1.18e-05   -4.68e-06
CapitalGain             9.776e-06   1.05e-07     93.318      0.000    9.57e-06    9.98e-06
CapitalLoss             9.952e-05    1.8e-06     55.394      0.000     9.6e-05       0.000
Dividends               1.552e-05   2.48e-07     62.566      0.000     1.5e-05     1.6e-05
INSTANCEWEIGHT          2.085e-06   4.89e-07      4.266      0.000    1.13e-06    3.04e-06
NumOfPersonForEmployer     0.0085      0.000     26.763      0.000       0.008       0.009
OwnBusiness               -0.0050      0.001     -5.476      0.000      -0.007      -0.003
VeteranBenefits           -0.0191      0.001    -19.676      0.000      -0.021      -0.017
WeeksWorked                0.0032   3.87e-05     83.109      0.000       0.003       0.003
Year                       0.0044      0.001      4.576      0.000       0.003       0.006
==============================================================================
Omnibus:                   119596.189   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           995579.191
Skew:                           2.882   Prob(JB):                         0.00
Kurtosis:                      12.302   Cond. No.                     8.95e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.95e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

We can see from the above result that none of the coefficients are zero, also some of the features have significant p-values, which indicates that there is a significant relationship among the predictors and the response. 

* Hence we reject our Null Hypothesis $H_0$.

We should keep in mind that we have not considered all the features for our hypothesis generation, we will explore more about the nominal features as we proceed in the coming sections.

## Baseline

In order to evaluate our model we should define some baseline. Let us generate some statistics about our response variable so that we can set our baseline.

In [5]:
# Getting the count
incomeCount = censusDf['targetIncome'].value_counts()
print(incomeCount)

# Getting the proportion of data having -50000 as response
print(float(incomeCount[0]/len(censusDf['targetIncome']))*100,
     "% people have income below $50000.")

0    187141
1     12382
Name: targetIncome, dtype: int64
93.79419916500854 % people have income below $50000.


Most of the values are 0 in the responce variable, Income. Which means that the dataset is heavily skewed towards having income less than \$50,000. Which means that if we predict only below \$50,000, still our model accuracy would be **93.79%**.

---

## Data Wrangling

### 1. Missing Value Imputation

In [6]:
censusDf.isnull().sum().sort_values(ascending=False).head()

HispanicOrigin    874
targetIncome        0
Race                0
Dividends           0
CapitalLoss         0
dtype: int64

* We can observe from the above statistics that, there are no missing values in numerical columns of the dataset. 
* There is only one column in which there are 874 missing values, which is 'HispanicOrigin'.
* From the first five lines of dataframe displayed above we saw that there are some garbage/missing values in the dataframe labelled as '?', lets try to track them.

In [7]:
# There are lot of '?' appearing in the dataset lets track them
for i in censusDf.columns:
    if '?' in list(censusDf[i]):
        print(censusDf.loc[censusDf[i].isin(['?'])][i].value_counts())

?    708
Name: PrevState, dtype: int64
?    99696
Name: MigrationCode_MSA, dtype: int64
?    99696
Name: MigrationCode_REG, dtype: int64
?    99696
Name: MigrationCode_WITHIN_REG, dtype: int64
?    99696
Name: MigrationPrevResInSunbelt, dtype: int64
?    6713
Name: BirthCountryFather, dtype: int64
?    6119
Name: BirthCountryMother, dtype: int64
?    3393
Name: BirthCountrySelf, dtype: int64


The above missing values does not makes much sense if we substitute them, as they are nominal values. Let us label all the above missing values as 'Unavailable'. Also there are four columns in which there almost 50% of the values which are '?', it is better to drop those columns, as high proportion of missing values can be misleading.

In [8]:
# Dropping the columns with missing values more than 50% and storing in a new dataframe
censusDf_cleaned = censusDf.drop(['MigrationCode_MSA', 'MigrationCode_REG', 
                                  'MigrationCode_WITHIN_REG', 
                                  'MigrationPrevResInSunbelt'], axis=1)

# Replacing the '?' with the label 'Unavailable'
censusDf_cleaned = censusDf_cleaned.replace('?', 'Unavailable')

In [9]:
# Check if the values are replaced
for i in censusDf_cleaned.columns:
    if 'Unavailable' in list(censusDf_cleaned[i]):
        print(censusDf_cleaned.loc[censusDf_cleaned[i].isin(['Unavailable'])][i].value_counts())

Unavailable    708
Name: PrevState, dtype: int64
Unavailable    6713
Name: BirthCountryFather, dtype: int64
Unavailable    6119
Name: BirthCountryMother, dtype: int64
Unavailable    3393
Name: BirthCountrySelf, dtype: int64


* As we saw earlier, for the caolumn 'HispanicOrigin' we have few (874) missing values; lets see how the values are distributed in the column, so that we can impute the missing values.

In [10]:
censusDf_cleaned['HispanicOrigin'].value_counts().sort_values(ascending=False)

All other                    171907
Mexican-American               8079
Mexican (Mexicano)             7234
Central or South American      3895
Puerto Rican                   3313
Other Spanish                  2485
Cuban                          1126
Do not know                     306
Chicano                         304
Name: HispanicOrigin, dtype: int64

Most of the values (86%) is 'All other', its better to replace the missing values with this one, as all other values are in insignificant numbers.

In [11]:
# store the missing value in a variable
missing_val = censusDf_cleaned[censusDf_cleaned.isnull()]['HispanicOrigin'].iloc[1]
# impute the missing values
censusDf_cleaned['HispanicOrigin'] = censusDf_cleaned['HispanicOrigin'].replace(
    missing_val, 'All other')

##### **Check for missing values one last time.**

In [12]:
# Check for missing values
censusDf_cleaned.isnull().sum().sort_values(ascending=False).head()

targetIncome          0
OccupationCode        0
CapitalGain           0
EmploymentStatus      0
UnemploymentReason    0
dtype: int64

> Now there are no missing values in the dataset.

### 2. Feature Engineering

In [13]:
# Categorizing the columns

# Continuous Features
ordinalFeatures = ['Age', 'WagePerHr', 'CapitalGain', 'CapitalLoss','Dividends', 
     'INSTANCEWEIGHT', 'NumOfPersonForEmployer', 'WeeksWorked']

# Nominal Features
nominalFeatures = ['ClassOfWorker', 'Industry', 'Occupation', 'Education', 
                  'EducationalInst', 'MaritalStatus', 'IndustryCode', 'OccupationCode',
                  'Race', 'HispanicOrigin', 'Sex', 'MemLabourUnion', 
                  'UnemploymentReason', 'EmploymentStatus','FEDERALTAX', 
                   'TaxFilerStat', 'PrevState', 'HouseholdStatus', 
                  'HouseholdSummary', 'LiveInHouse',
                  'Parent', 'BirthCountryFather', 'BirthCountryMother',
                  'BirthCountrySelf', 'Citizenship', 'OwnBusiness', 'VeteranQA', 'VeteranBenefits', 
                  'Year', 'targetIncome']

# Replacing the 'targetIncome' values with dummy variables
# - 50000. as the baseline. 0 for - 50000. and 1 for 50000+.
censusDf_cleaned['targetIncome'] = pd.get_dummies(censusDf_cleaned.targetIncome).iloc[:,1:]

# Check the features
print(len(censusDf_cleaned.columns) == len(ordinalFeatures) + len(nominalFeatures))

# Features and Outcome
X = censusDf_cleaned.drop('targetIncome',1)
Y = censusDf_cleaned.targetIncome
print("X (predictors) is ",X.shape[0],"rows,", X.shape[1],"columns, and..."\
      "\nY (response) is ",Y.shape[0],"rows.")


True
X (predictors) is  199523 rows, 37 columns, and...
Y (response) is  199523 rows.


##### **Let us check the categorical variables in for each feature, and decide which one to  use in our model.**

In [14]:
# Print out number of unique categorical values in each column
print("NUMBER OF UNIQUE VALUES IN EACH FEATURE:\n")
for col_name in X.columns:
    if X[col_name].dtype == 'object':
        unique_val = len(X[col_name].unique())
        print("'{col_name}' has --> {unique_val}\
        ".format(col_name=col_name, unique_val=unique_val))

NUMBER OF UNIQUE VALUES IN EACH FEATURE:

'ClassOfWorker' has --> 9        
'Education' has --> 17        
'EducationalInst' has --> 3        
'MaritalStatus' has --> 7        
'IndustryCode' has --> 24        
'OccupationCode' has --> 15        
'Race' has --> 5        
'HispanicOrigin' has --> 9        
'Sex' has --> 2        
'MemLabourUnion' has --> 3        
'UnemploymentReason' has --> 6        
'EmploymentStatus' has --> 8        
'FEDERALTAX' has --> 6        
'TaxFilerStat' has --> 6        
'PrevState' has --> 51        
'HouseholdStatus' has --> 38        
'HouseholdSummary' has --> 8        
'HouseOneYearAgo' has --> 3        
'Parent' has --> 5        
'BirthCountryFather' has --> 43        
'BirthCountryMother' has --> 43        
'BirthCountrySelf' has --> 43        
'Citizenship' has --> 5        
'VeteranQA' has --> 3        


##### It looks like the columns 'BirthCountryFather', 'BirthCountryMother' and 'BirthCountrySelf' have same number of unique values. Let us keep only one column, and drop the other two.

In [15]:
# Dropping the columns
X = X.drop(['BirthCountryFather', 'BirthCountryMother'], axis=1)
# keeping 'BirthCountrySelf' and renaming
X.rename(columns={'BirthCountrySelf': 'BirthCountry'}, inplace=True)

In [16]:
# Although, 'BirthCountry' has a lot of unique categories, ...
# ...most categories only have a few observations if compared to max (United-States)
X['BirthCountry'].value_counts().sort_values(ascending=False).head(10)

United-States         176989
Mexico                  5767
Unavailable             3393
Puerto-Rico             1400
Germany                  851
Philippines              845
Cuba                     837
Canada                   700
Dominican-Republic       690
El-Salvador              689
Name: BirthCountry, dtype: int64

In [17]:
# In this case, bucket low frequecy categories as "Other"
X['BirthCountry'] = ['United-States' if x == 'United-States' 
                       else 'Other-Countries' for x in X['BirthCountry']]
# check the values
X['BirthCountry'].value_counts().sort_values(ascending=False)

United-States      176989
Other-Countries     22534
Name: BirthCountry, dtype: int64

##### The column 'HouseholdStatus' has 38 unique values; only few of the categories have significant number of observations.

In [18]:
# Check the value counts
X['HouseholdStatus'].value_counts().sort_values(ascending=False).head(10)

Householder                                        53248
Child <18 never marr not in subfamily              50326
Spouse of householder                              41695
Nonfamily householder                              22213
Child 18+ never marr Not in a subfamily            12030
Secondary individual                                6122
Other Rel 18+ ever marr not in subfamily            1956
Grandchild <18 never marr child of subfamily RP     1868
Other Rel 18+ never marr not in subfamily           1728
Grandchild <18 never marr not in subfamily          1066
Name: HouseholdStatus, dtype: int64

It is better to categorize the values as other, which does not have significant count.

In [19]:
# Bucket the low frequency category as other
X['HouseholdStatus'] = ['Householder' if x == 'Householder'
                        else 'Children' if x == 'Child <18 never marr not in subfamily'
                        else 'Spouse' if x == 'Spouse of householder'
                        else 'Nonfamily' if x == 'Nonfamily householder'
                        else 'Child_18_plus' if x == 'Child 18+ never marr Not in a subfamily'
                        else 'Secondary_indv' if x == 'Secondary individual'
                       else 'Other_Householders' for x in X['HouseholdStatus']]
# check the values
X['HouseholdStatus'].value_counts().sort_values(ascending=False)

Householder           53248
Children              50326
Spouse                41695
Nonfamily             22213
Other_Householders    13889
Child_18_plus         12030
Secondary_indv         6122
Name: HouseholdStatus, dtype: int64

##### Lets check the 'PrevState' column, there are 51, unique values for the feature, lets see what are they.

In [20]:
# Check the value counts
X['PrevState'].value_counts().sort_values(ascending=False).head(10)

Not in universe    183750
California           1714
Utah                 1063
Florida               849
North Carolina        812
Unavailable           708
Abroad                671
Oklahoma              626
Minnesota             576
Indiana               533
Name: PrevState, dtype: int64

With approximately 200,000 rows in our dataset, there are almost 184,000 values for the 'PrevState' column, that say 'Not in universe', which is almost 96% of the entire row, have this much small information about the sate doesn't seem to be helpful, it is better that we drop this feature from our predictors variables list.

In [21]:
# Dropping the 'PrevState' column
X = X.drop(['PrevState'], axis=1)

#### Creating Dummies

**Coverting categorical variable in to _Dummy Variables_.** If we want to include a categorical feature in our machine learning model, one common solution is to create dummy variables. We drop the original feature from the dataset and add a dummied version of the feature to the dataset, which is easier for the model to interpret.

In [22]:
# Creating a list of categorical features to create a dummy variable of
features_to_dummy = ['NumOfPersonForEmployer', 'OwnBusiness', 'VeteranBenefits', 
                     'Year'] + ['ClassOfWorker', 'Education', 'EducationalInst', 
                                'MaritalStatus', 
                     'IndustryCode', 'OccupationCode', 'Race', 
                     'HispanicOrigin', 'Sex', 'MemLabourUnion', 'UnemploymentReason', 
                     'EmploymentStatus', 'FEDERALTAX', 'TaxFilerStat', 
                     'HouseholdStatus', 'HouseholdSummary', 'HouseOneYearAgo', 
                     'Parent', 'BirthCountry', 'Citizenship', 'VeteranQA']

Define a function to create dummy variables of the dataframe from the list of columns.

In [23]:
# Function to create the dummy categorical variables used for modeling
def create_dummies(df, col_name_list):
    """
    This function takes the dataframe and features list as input, 
    and returns the modified dataframe with dummy variables of the 
    features in the list col_name_list.
    
    :param df: target dataframe 
    :param col_name_list: list of the column names from the dataset
    :return: modifies the dataframe df inplace and returns dummied dataframe
             of features in col_name_list
    """
    for x in col_name_list:
        dummies = pd.get_dummies(df[x], prefix=x, dummy_na=False)
        df = df.drop(x, 1)
        df = pd.concat([df, dummies], axis=1)
    return df

Calling the function create_dummies to convert our features in to dummy variables.

In [24]:
# Before dummies
print("Dataframe X has", X.shape[1],"columns",X.shape[0],"and rows.")

# Call the function create_dummies on X and replace the features with dummies
print("Creating dummies ...")
X = create_dummies(X, features_to_dummy)

# Printing the dimensions of the modified feature set
print("*** Now our dataframe has", X.shape[1],"columns",X.shape[0],"and rows. ***")

# display first five rows of all the features
with pd.option_context('display.max_columns', None):
    display(X.head())

Dataframe X has 34 columns 199523 and rows.
Creating dummies ...
*** Now our dataframe has 177 columns 199523 and rows. ***


,Age,Industry,Occupation,WagePerHr,CapitalGain,CapitalLoss,Dividends,INSTANCEWEIGHT,WeeksWorked,NumOfPersonForEmployer_0,NumOfPersonForEmployer_1,NumOfPersonForEmployer_2,NumOfPersonForEmployer_3,NumOfPersonForEmployer_4,NumOfPersonForEmployer_5,NumOfPersonForEmployer_6,OwnBusiness_0,OwnBusiness_1,OwnBusiness_2,VeteranBenefits_0,VeteranBenefits_1,VeteranBenefits_2,Year_94,Year_95,ClassOfWorker_Federal government,ClassOfWorker_Local government,ClassOfWorker_Never worked,ClassOfWorker_Not in universe,ClassOfWorker_Private,ClassOfWorker_Self-employed-incorporated,ClassOfWorker_Self-employed-not incorporated,ClassOfWorker_State government,ClassOfWorker_Without pay,Education_10th grade,Education_11th grade,Education_12th grade no diploma,Education_1st 2nd 3rd or 4th grade,Education_5th or 6th grade,Education_7th and 8th grade,Education_9th grade,Education_Associates degree-academic program,Education_Associates degree-occup /vocational,Education_Bachelors degree(BA AB BS),Education_Children,Education_Doctorate degree(PhD EdD),Education_High school graduate,Education_Less than 1st grade,Education_Masters degree(MA MS MEng MEd MSW MBA),Education_Prof school degree (MD DDS DVM LLB JD),Education_Some college but no degree,EducationalInst_College or university,EducationalInst_High school,EducationalInst_Not in universe,MaritalStatus_Divorced,MaritalStatus_Married-A F spouse present,MaritalStatus_Married-civilian spouse present,MaritalStatus_Married-spouse absent,MaritalStatus_Never married,MaritalStatus_Separated,MaritalStatus_Widowed,IndustryCode_Agriculture,IndustryCode_Armed Forces,IndustryCode_Business and repair services,IndustryCode_Communications,IndustryCode_Construction,IndustryCode_Education,IndustryCode_Entertainment,IndustryCode_Finance insurance and real estate,IndustryCode_Forestry and fisheries,IndustryCode_Hospital services,IndustryCode_Manufacturing-durable goods,IndustryCode_Manufacturing-nondurable goods,IndustryCode_Medical except hospital,IndustryCode_Mining,IndustryCode_Not in universe or children,IndustryCode_Other professional services,IndustryCode_Personal services except private HH,IndustryCode_Private household services,IndustryCode_Public administration,IndustryCode_Retail trade,IndustryCode_Social services,IndustryCode_Transportation,IndustryCode_Utilities and sanitary services,IndustryCode_Wholesale trade,OccupationCode_Adm support including clerical,OccupationCode_Armed Forces,OccupationCode_Executive admin and managerial,OccupationCode_Farming forestry and fishing,OccupationCode_Handlers equip cleaners etc,OccupationCode_Machine operators assmblrs & inspctrs,OccupationCode_Not in universe,OccupationCode_Other service,OccupationCode_Precision production craft & repair,OccupationCode_Private household services,OccupationCode_Professional specialty,OccupationCode_Protective services,OccupationCode_Sales,OccupationCode_Technicians and related support,OccupationCode_Transportation and material moving,Race_Amer Indian Aleut or Eskimo,Race_Asian or Pacific Islander,Race_Black,Race_Other,Race_White,HispanicOrigin_All other,HispanicOrigin_Central or South American,HispanicOrigin_Chicano,HispanicOrigin_Cuban,HispanicOrigin_Do not know,HispanicOrigin_Mexican (Mexicano),HispanicOrigin_Mexican-American,HispanicOrigin_Other Spanish,HispanicOrigin_Puerto Rican,Sex_Female,Sex_Male,MemLabourUnion_No,MemLabourUnion_Not in universe,MemLabourUnion_Yes,UnemploymentReason_Job leaver,UnemploymentReason_Job loser - on layoff,UnemploymentReason_New entrant,UnemploymentReason_Not in universe,UnemploymentReason_Other job loser,UnemploymentReason_Re-entrant,EmploymentStatus_Children or Armed Forces,EmploymentStatus_Full-time schedules,EmploymentStatus_Not in labor force,EmploymentStatus_PT for econ reasons usually FT,EmploymentStatus_PT for econ reasons usually PT,EmploymentStatus_PT for non-econ reasons usually FT,EmploymentStatus_Unemployed full-time,EmploymentStatus_Unemployed part- time,FEDERALTAX_Head of household,FEDERALTAX_Joi

#### Principal Components Analysis

Principal component analysis (PCA) transforms the dataset of many features into few Principal Components that "summarize" the variance underying in the data. It is the  most common way of dimensionality reduction, and it works well where the features are highly corelated. The drawback of using PCA is that it makes it difficult to interpret the data.

In [25]:
# We will use PCA from sklearn.decomposition to find the principal components
pca = PCA(n_components=10) # 10 principal components
X_pca = pd.DataFrame(pca.fit_transform(X))

In [26]:
# Displaying the first few rows of 10 pcs
X_pca.head()

,0,1,2,3,4,5,6,7,8,9
0,-446.996481,-168.033982,-40.410671,-65.496457,-13.481102,15.989761,-45.486705,-0.617677,1.094478,-1.122026
1,-447.339268,-167.875246,-686.890209,-61.332959,-12.915837,-32.694435,-15.842541,28.451046,-7.561815,0.985290
2,-447.408330,-167.895835,-748.541316,-62.672028,-13.051615,30.848877,7.465579,-0.582813,-0.705724,-1.419406
3,-446.981059,-168.121161,17.640878,-66.047630,-13.753165,34.065352,15.926933,-0.593192,-1.086352,1.036290
4,-447.367195,-167.926438,-671.331974,-63.046326,-13.147034,33.220744,15.177789,-0.592779,-1.011346,1.027940


_In this case we will not proceed with the principal components. Because it is not recommended to perform PCA on categorical data._

In [27]:
c

NameError: name 'c' is not defined

In [76]:
# Training set

# [height, weight, shoeSize] 
X = [[181, 80, 44], [177, 70, 43], [160, 60, 38], [154, 54, 37], [166, 65, 40], 
     [190, 90, 47], [175, 64, 39], [177, 70, 40], [159, 55, 37], [171, 75, 42], [181, 85, 43]]

# [male or female]
Y = [0,0,1,1,0,0,1,1,1,0,0]

### Model Building

In [77]:
# Classifiers
from sklearn.linear_model import LogisticRegression

# metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

In [171]:
class ModelMetrics(object):

    # Random permutation cross-validator with 80-20 train test split
    cv = ShuffleSplit(n_splits = 2, test_size = 0.2) # change this to 10
    # dictionary to store scores
    model_scores = defaultdict(lambda:None)

    def __init__(self, model_name, model_obj, features, response):
        self.model = model_name
        self.model_obj = model_obj
        ModelMetrics.model_scores[model_name] = []
        self.cv = ModelMetrics.cv
        self.features = features
        self.response = response
        self.model_scores = ModelMetrics.model_scores

    def model_scoring(self, scoring_metric):
        for metric in scoring_metric:
            n_fold_score = cross_val_score(self.model_obj,self.features,
                                                           self.response,
                                                           cv=self.cv,
                                                           scoring=metric)
            mean_score = np.mean(n_fold_score)
            self.model_scores[self.model].append({metric:mean_score})
        return model_scores

#### Logistic Regression Model

In [172]:
# Creating the classifiers

# Classifier implementing Logistic Regression
clf_log_reg = LogisticRegression()
# Creating object for metrics
log_reg_metrics_obj = ModelMetrics("Logistic Regression", clf_log_reg, X, Y)
log_reg_metrics = log_reg_metrics_obj.model_scoring(['accuracy','precision','recall'])

In [179]:
a

defaultdict(None,
            {'Logistic Regression': [{'accuracy': 1.0},
              {'precision': 1.0},
              {'recall': 0.5}]})

In [180]:
type(a)

collections.defaultdict

In [181]:
for key, val in a.items():
    print('key=',key,"\nVal=",val)

key= Logistic Regression 
Val= [{'accuracy': 1.0}, {'precision': 1.0}, {'recall': 0.5}]


---

## Future work

- Work on feature engineering, try to explore features that can help predicting the target more efficiently, come up with new columns.
- Data Wrangling: try to clean the data whereever possible, drop the column if required.
- Work on Hypothesis and baseline.
- Work on model building and fitting the data into the model. Use the concepts taught in the class in order to fit the model, few of the classifiers which we are planning to use:
    - Logistic Regression
    - Gaussian Naive Bayes
    - Concepts of resampling, for example Cross Validation
    - Decision Tree
    - Random Forest classifier
    - Linear Regression
    - Ridge/Lasso Regression
    - Support Vector Machines
- Create diagnostic tools for the models.
- Define a metric to compare the models.